In [1]:
import fasttext
import fasttext.util
import pandas as pd
import numpy as np

from keybert import KeyBERT

/lyceum/jhk1c21/.conda/envs/msc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-01 15:50:22.947135: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 15:51:13.890701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import os

In [ ]:
data_home = "/lyceum/jhk1c21/msc_project/data/"

In [ ]:
# df = pd.read_pickle(data_home + "graph/nodes_full.pkl")
df = pd.read_csv(os.path.join(data_home, "graph", "full", "nodes_full.csv"), index_col='_id')

In [8]:
# title_list = list(df['title'])
# keywords_list = list(df['keywords'])
abstract_list = df['abstract']
# fos_list = list(df['fos'])

In [21]:
abstract_dict = df['abstract'].to_dict()
abstract_dict

{'53e99784b7602d9701f3f411': 'The eXtensible Markup Language 驴 XML 驴 is not only a language for communication between humans and the web, it is also a language for communication between programs. Rather than passing parameters, programs can pass documents from one to another, containing not only pure data, but control information as well. Even legacy programs written in ancient languages such as COBOL and PL/I can be adapted by means ofinterface reengineering to process and to generate XML documents.',
 '53e99784b7602d9701f3f5fe': 'Resource allocation for multi-tier web applications in virtualization environments is one of the most important problems in autonomous computing. On one hand, the more resources that are provisioned to a multitier web application, the easier it is to meet service level objectives (SLO). On the other hand, the virtual machine which hosts the multi-tier web application needs to be consolidated as much as possible in order to maintain high resource utilization.

In [12]:
title_list = np.load(data_home + "graph/title.npy")
keywords_list = np.load(data_home + "graph/keywords.npy")
abstract_list = np.load(data_home + "graph/abstract.npy")
fos_list = np.load(data_home + "graph/fos.npy")

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:
fast_model = fasttext.load_model('./data/cc.en.300.bin')

In [ ]:
def word_embedding(model, keywords_list):
    embeddings = [ np.array(list(map(model.get_word_vector, keywords))) for keywords in keywords_list ]
    return embeddings
    
def word_list_embedding(model, keywords_list):
    embeddings = word_embedding(model, keywords_list)
    # print(len(embeddings))
    # return np.array([ np.mean(keywords_embedding, axis=0) for keywords_embedding in embeddings])
    return [ np.mean(keywords_embedding, axis=0) for keywords_embedding in embeddings]
    
def sentence_embedding(model, sentence_list):
    return np.array([ model.get_sentence_vector(' '.join(str(sentence).split())) for sentence in sentence_list ])

In [ ]:
keywords_embedding_list = word_list_embedding(fast_model, keywords_list)

In [ ]:
title_embedding_list = sentence_embedding(fast_model, title_list)

In [31]:
keywords_from_abstract = []
threshold = 0.3
keywords_limit = 2
# bert_model = KeyBERT('distilbert-base-nli-mean-tokens')
bert_model = KeyBERT()

In [59]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Ensure the model is in evaluation mode
model.eval()

# If you have a GPU, put the model on GPU
if torch.cuda.is_available():
    model.to('cuda')

def extract_keywords(text, top_n=5):
    """
    Extract top_n keywords from a text using DistilBERT embeddings.
    """
    # Tokenize input and get outputs from DistilBERT
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    with torch.no_grad():
        if torch.cuda.is_available():
            inputs = {key: val.to('cuda') for key, val in inputs.items()}
        outputs = model(**inputs)
    
    # Get the embeddings of the tokens
    embeddings = outputs.last_hidden_state[0]
    
    # Compute norms of the embeddings
    norms = torch.norm(embeddings, dim=1).cpu().numpy()
    
    # Filter out "unused" tokens and rank remaining tokens
    filtered_tokens = [(word, norm) for word, norm in zip(tokenized_words, norms) if "unused" not in word]
    sorted_tokens = sorted(filtered_tokens, key=lambda x: x[1], reverse=True)
    
    # Get the top_n keywords
    keywords = [token[0] for token in sorted_tokens[:top_n]]
    
    return keywords

In [67]:
df['abstract'].to_numpy()

array(['The eXtensible Markup Language 驴 XML 驴 is not only a language for communication between humans and the web, it is also a language for communication between programs. Rather than passing parameters, programs can pass documents from one to another, containing not only pure data, but control information as well. Even legacy programs written in ancient languages such as COBOL and PL/I can be adapted by means ofinterface reengineering to process and to generate XML documents.',
       'Resource allocation for multi-tier web applications in virtualization environments is one of the most important problems in autonomous computing. On one hand, the more resources that are provisioned to a multitier web application, the easier it is to meet service level objectives (SLO). On the other hand, the virtual machine which hosts the multi-tier web application needs to be consolidated as much as possible in order to maintain high resource utilization. This paper presents an adaptive resource co

In [69]:
keywords = bert_model.extract_keywords(df['abstract'].to_numpy()[:3], top_n=10, keyphrase_ngram_range=(1, 2), use_mmr=True, diversity=0.6, use_maxsum=False)
keywords

[[('xml language', 0.749),
  ('extensible markup', 0.4908),
  ('ofinterface reengineering', 0.3282),
  ('communication humans', 0.2729),
  ('data control', 0.2657),
  ('legacy programs', 0.2539),
  ('passing parameters', 0.1818),
  ('process generate', 0.1781),
  ('written ancient', 0.1736),
  ('cobol pl', 0.1389)],
 [('applications virtualization', 0.5221),
  ('adaptive resource', 0.4638),
  ('service qos', 0.3697),
  ('data center', 0.2667),
  ('improve cpu', 0.2563),
  ('tier', 0.2285),
  ('xen', 0.2178),
  ('controller consists', 0.1567),
  ('slo hand', 0.1181),
  ('average', 0.1079)],
 [('circuit timing', 0.6101),
  ('carlo simulation', 0.2605),
  ('register paths', 0.237),
  ('sub micron', 0.2264),
  ('shift deterministic', 0.2097),
  ('reconvergence correlations', 0.1641),
  ('built current', 0.1575),
  ('yield results', 0.1195),
  ('design methods', 0.0978),
  ('average errors', 0.0721)]]

In [35]:
for idx, abstract in enumerate(list(abstract_dict.values())):
    # print(id, abstract)
    keywords = bert_model.extract_keywords(abstract, top_n=10, keyphrase_ngram_range=(1, 2), use_mmr=True, diversity=0.6, use_maxsum=False)
    print(keywords)
    filtered_keywords = [ keyword[0] for keyword in keywords if keyword[1] > threshold ]
    if len(filtered_keywords) == 0:
        filtered_keywords = [ keyword[0] for keyword in keywords[:keywords_limit] ]
    
    keywords_from_abstract.append(filtered_keywords)
    if idx % 10 == 0:
        print(f"{idx} is done => shape: {len(filtered_keywords)}")

53e99784b7602d9701f3f411 The eXtensible Markup Language 驴 XML 驴 is not only a language for communication between humans and the web, it is also a language for communication between programs. Rather than passing parameters, programs can pass documents from one to another, containing not only pure data, but control information as well. Even legacy programs written in ancient languages such as COBOL and PL/I can be adapted by means ofinterface reengineering to process and to generate XML documents.


[('xml language', 0.749), ('extensible markup', 0.4908), ('ofinterface reengineering', 0.3282), ('communication humans', 0.2729), ('data control', 0.2657), ('legacy programs', 0.2539), ('passing parameters', 0.1818), ('process generate', 0.1781), ('written ancient', 0.1736), ('cobol pl', 0.1389)]
0 is done => shape: 3
53e99784b7602d9701f3f5fe Resource allocation for multi-tier web applications in virtualization environments is one of the most important problems in autonomous computing. On one hand, the more resources that are provisioned to a multitier web application, the easier it is to meet service level objectives (SLO). On the other hand, the virtual machine which hosts the multi-tier web application needs to be consolidated as much as possible in order to maintain high resource utilization. This paper presents an adaptive resource controller which consists of a feedback utilization controller and an auto-regressive and moving average model (ARMA)-based model estimator. It can mee

KeyboardInterrupt: 

In [ ]:
for idx, abstract in enumerate(abstract_list):
    keywords = bert_model.extract_keywords(abstract, top_n=20, use_mmr=True)
    
    filtered_keywords = [ keyword[0] for keyword in keywords if keyword[1] > threshold ]
    if len(filtered_keywords) == 0:
        filtered_keywords = [ keyword[0] for keyword in keywords[:keywords_limit] ]
    
    keywords_from_abstract.append(filtered_keywords)
    if idx % 10_000 == 0:
        print(f"{idx} is done => shape: {len(filtered_keywords)}")

In [ ]:
abstract_embedding_list = word_list_embedding(fast_model, keywords_from_abstract)

In [ ]:
to_eleminate_list = []
original_shape = abstract_embedding_list[0].shape
for idx, embedding in enumerate(abstract_embedding_list):
    if original_shape != embedding.shape:
        to_eleminate_list.append(idx)
        print(idx, len(abstract_list[idx]), abstract_list[idx])

In [ ]:
keywords_embedding = []
title_embedding = []
abstract_embedding = []

filtered_fos = []
filtered_id = []
start = 0

for end in to_eleminate_list:
    keywords_embedding.extend(keywords_embedding_list[start:end])
    title_embedding.extend(title_embedding_list[start:end])
    abstract_embedding.extend(abstract_embedding_list[start:end])
    
    filtered_fos.extend(list(df['fos'].iloc[:][start:end]))
    filtered_id.extend(list(df['_id'].iloc[:][start:end]))
    start = end + 1
    
keywords_embedding.extend(keywords_embedding_list[start:])
title_embedding.extend(title_embedding_list[start:])
abstract_embedding.extend(abstract_embedding_list[start:])

filtered_fos.extend(list(df['fos'].iloc[:][start:]))
filtered_id.extend(list(df['_id'].iloc[:][start:]))

In [ ]:
keywords_embedding = np.array(keywords_embedding)
title_embedding = np.array(title_embedding)
abstract_embedding = np.array(abstract_embedding)

filtered_id = np.array(filtered_id)
filtered_fos = np.array(filtered_fos, dtype='object')

In [ ]:
np.save('/data/jhk1c21/graph_data/title_full.npy', title_embedding)
np.save('/data/jhk1c21/graph_data/keywords_full.npy', keywords_embedding)
np.save('/data/jhk1c21/graph_data/abstract_full.npy', abstract_embedding)
np.save('/data/jhk1c21/graph_data/id_full.npy', filtered_id)
np.save('/data/jhk1c21/graph_data/fos_full.npy', filtered_fos)

In [ ]:
np.save('/data/jhk1c21/graph_data/title_full.npy', np.array(title_embedding_list))
np.save('/data/jhk1c21/graph_data/keywords_full.npy', np.array(keywords_embedding_list))